In [11]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *

# アプリ名の設定
AppSettings.setCurrentApp("SUITE2P_ROI_CHECK")

class Suite2pROICheckGUI(QMainWindow, WidgetManager):
    def __init__(self):
        """
        pri: 格納するkey
        """
        QMainWindow.__init__(self)
        WidgetManager.__init__(self)
        # self.class_HoloStimResponse = None

        # Tableの列名と列番号, 列情報のdict
        self.dict_tablecol = AppSettings.getTableColumns()
        # キーと機能のマッピング
        self.key_function_map = AppSettings.getKeyFunctionMap()
        # GUIのコンフィグ
        self.gui_defaults = AppSettings.getGuiDefaults()

        self.setWindowTitle(self.gui_defaults["TITLE"])
        self.setGeometry(self.gui_defaults["INIT_POSITION_X"], self.gui_defaults["INIT_POSITION_Y"], self.gui_defaults["WIDTH"], self.gui_defaults["HEIGHT"])
        self.setupUI_done = False
        
        # self.initializeAttributes()

        self.setupFileLoadUI()

    """
    setup UI Function
    """
    def setupFileLoadUI(self):
        self.mainLayout = self.makeLayoutMain()
        self.mainLayout.addLayout(self.makeLayoutSectionBottom(), 1, 0, 1, 3)
        
        # bindFunc
        self.bindFuncFileLoadUI()

    # setup
    def setupUI(self):
        self.mainLayout.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        self.mainLayout.addLayout(self.makeLayoutSectionMiddleUpper(), 0, 1, 1, 1)
        self.mainLayout.addLayout(self.makeLayoutSectionRightUpper(), 0, 2, 1, 1)

        # bindFunc
        self.bindFuncAllWidget()

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    # Main Layout
    def makeLayoutMain(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        return QGridLayout(central_widget)

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)", "Cellpose Mask path (optional)"]
        list_key = [f"path_fall_{app_keys}", f"path_reftif_{app_keys}", f"path_cellpose_{app_keys}"]
        for label, key in zip(list_label, list_key):
            layout.addLayout(makeLayoutLoadFileWidget(self, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout_button = QHBoxLayout()
        list_label = ["Load Files", "Exit", "Help"]
        list_key = ["load_file", "exit", "help"]
        
        for label, key in zip(list_label, list_key):
            layout_button.addWidget(self.makeWidgetButton(key=key, label=label))
        layout.addLayout(layout_button)
        return layout

    "Left Upper"
    # EventFileの読み込み, plot用
    def makeLayoutComponentEventFilePlot(self):
        layout = QVBoxLayout()
        # Eventのplot range
        layout.addLayout(makeLayoutLineEditLabel(self,
                                                 key_label="eventfile_align_plot_range",
                                                 key_lineedit="eventfile_align_plot_range",
                                                 label="plot range from Event start (pre, post; sec)",
                                                 text_set="(10, 10)"))
        # プロットに重ねるEventFile npyファイルの読み込みボタン, Clearボタン
        layout.addWidget(self.makeWidgetButton(key="loadEventFile", label="Load EventFile npy file"))  ### 要修正
        layout.addWidget(self.makeWidgetButton(key="clearEventFile", label="Clear"))
        return layout

    # trace plotの表示範囲
    def makeLayoutComponentTracePlotRange(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutLineEditLabel(self,
                                                key_label="minPlotRange",
                                                key_lineedit="minPlotRange",
                                                label="Minimum plot range (sec)",
                                                text_set="30"))
        layout.addWidget(self.makeWidgetCheckBox(key="plot_eventfile_trace", 
                                                 label="plot EventFile trace", 
                                                 checked=True,
                                                 ))
        return layout

    # 上記二つを合体
    def makeLayoutComponentEventFilePlot_TracePlotRange(self):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentEventFilePlot())
        layout.addLayout(self.makeLayoutComponentTracePlotRange())
        return layout

    "Middle Upper"
    # ROI canvas
    def makeLayoutComponentROIView(self):
        layout = QVBoxLayout()
        layout.addWidget(self.makeWidgetView(key=self.gui_defaults["APP_KEYS"]))
        return layout

    # ROI property label, threshold lineedit
    def makeLayoutComponentROIPropertyDisplay_Threshold(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        layout.addLayout(makeLayoutROIProperty(self, key_label=f"{app_keys}_roi_prop"))
        layout.addLayout(makeLayoutROIThresholds(
            self, 
            key_label=f"{app_keys}_roi_threshold", 
            key_lineedit=f"{app_keys}_roi_threshold", 
            key_checkbox=f"{app_keys}_roi_threshold", 
            label_checkbox="ROI Show Threshold", 
            list_threshold_param=["npix", "compact"]))
        return layout

    # ROI display, background image button group, checkbox
    def makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip(self):
        layout = QVBoxLayout()
        makeLayoutROITypeDisplay(widget_manager, key_buttongroup)
        makeLayoutBGImageTypeDisplay(widget_manager, key_buttongroup)
        makeLayoutROIChooseSkip(widget_manager, key_checkbox)

    # channel contrast, ROI opacity slider
    def makeLayoutComponentContrastOpacitySlider(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        layout.addLayout(makeLayoutContrastSlider(self, 
                                                key_label=app_keys, 
                                                key_checkbox=app_keys,
                                                key_slider=app_keys,
                                                label_checkbox=f"Show {app_keys} channel", 
                                                label_label=f"{app_keys} Value", 
                                                checked=True))
        layout.addLayout(makeLayoutOpacitySlider(self, 
                                                key_label=app_keys, 
                                                key_slider=app_keys, 
                                                label=app_keys
                                                ))
        return layout

    "Right Upper"

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 左上
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutCanvasTracePlot(self, 
                                                   key_figure=self.gui_defaults["APP_KEYS"], 
                                                   key_canvas=self.gui_defaults["APP_KEYS"], 
                                                   key_app=self.gui_defaults["APP_KEYS"]), 
                        stretch=1)
        layout.addLayout(self.makeLayoutComponentEventFilePlot_TracePlotRange())
        return layout

    # 中上
    def makeLayoutSectionMiddleUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIView())
        layout.addLayout(self.makeLayoutComponentROIPropertyDisplay_Threshold())
        layout.addLayout(self.makeLayoutComponentContrastOpacitySlider())
        return layout
    
    # 右上
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        return layout

    # 下
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):        
        app_keys = self.gui_defaults["APP_KEYS"]
        list_key = [f"path_fall_{app_keys}", f"path_reftif_{app_keys}", f"path_cellpose_{app_keys}"]
        list_filetype = ["mat", "tiff", "npy"]
        for key, filetype in zip(list_key, list_filetype):
            bindFuncLoadFileWidget(q_widget=self, q_button=self.dict_button[key], q_lineedit=self.dict_lineedit[key], filetype=filetype)

        list_key = ["load_file", "exit", "help"]
        list_func_ = [lambda: self.setupUI(), lambda: exitApp(self), lambda: exitApp(self)]
        for key, func_ in zip(list_key, list_func_):
            self.dict_button[key].clicked.connect(func_)

    def bindFuncAllWidget(self):
        app_keys = self.gui_defaults["APP_KEYS"]


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

In [5]:
gui.dict_ax

{'pri_top': <Axes: >, 'pri_middle': <Axes: >, 'pri_bottom': <Axes: >}